In [1]:
import src.get_data as get_data
import src.load_data as load
import src.indicators as ind

import pandas as pd
import seaborn as sns
from pylab import rcParams
import numpy as np
import matplotlib.pyplot as plt
import sqlalchemy

%matplotlib inline

sns.set(style='whitegrid', palette='muted', font_scale=1.5)

rcParams['figure.figsize'] = 14, 8

# some magic
%load_ext autoreload
%autoreload 2

In [2]:
# Research data-depend by indicators for ETH

df = get_data.get_data_frame('USDT_ETH', 'poloniex' )

features_for_analise = df[[ 'open','close','low','high','volume']].copy()

In [3]:
# compute indicators

features_for_analise = ind.ma_rel_diff(features_for_analise)
features_for_analise = ind.ema_rel_diff(features_for_analise)
features_for_analise = ind.mom(features_for_analise)
features_for_analise = ind.roc(features_for_analise)
features_for_analise = ind.bbands(features_for_analise)
features_for_analise = ind.normalized_bbands(features_for_analise)
features_for_analise = ind.rsi(features_for_analise)
features_for_analise = ind.stochastics(features_for_analise)
features_for_analise = ind.macd(features_for_analise)
features_for_analise = ind.atr(features_for_analise)
features_for_analise = ind.adx(features_for_analise)

In [4]:
features_for_analise = features_for_analise.dropna()

In [5]:
from sklearn.preprocessing import MinMaxScaler
import plotly as py
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot

init_notebook_mode(connected=True)

scaler = MinMaxScaler()

sc = scaler.fit_transform(features_for_analise)
sc_dataframe =  pd.DataFrame(sc,columns=features_for_analise.columns, index=features_for_analise.index)     

features = ['open', 'low', 'high', 'volume', 'ma_rel_diff_50',
       'ema_rel_diff_10', 'moment_20', 'roc_14', 'boil_mean_20_2',
       'boil_up_20_2', 'boil_down_20_2', 'normBB', 'rsi_14', 'stoch_d_14_3',
       'macd_12_26_9', 'atr_14', 'ATR', 'adx_14']

In [ ]:
# plot

for i in col:
    featureValues = go.Scatter( x = sc_dataframe.index, y = sc_dataframe[ i].values, name = i)
    closeValues = go.Scatter( x = sc_dataframe.index, y = sc_dataframe.close.values, name = 'close')

    iplot([featureValues,closeValues])
    plt.show()

In [96]:
# Feature SELECTING
"""from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LassoCV

x = features_for_analise[features].copy()
y = features_for_analise['close'].shift(1).copy()
y = y.dropna()

x_scaler = MinMaxScaler()
y_scaler = MinMaxScaler()

x[features] = x_scaler.fit_transform(x)
y = y_scaler.fit_transform(y.values.reshape(-1, 1))

num_train = round(0.9*x.shape[0])
train = x[:num_train]
test = x[num_train:]

# Keep only the most contributing features
sfm = SelectFromModel(LassoCV())
sfm.fit(train, y_train[:num_train])
train = sfm.transform(train)
test = sfm.transform(test)"""
print()

In [6]:
from keras.callbacks import History 
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential, load_model, save_model
from keras import backend as K 
import time
import datetime
from sklearn import metrics

WINDOW = 22

Using TensorFlow backend.


Couldn't import dot_parser, loading of dot files will not be possible.


In [7]:
def build_model(input_shape):
    d = 0.2
    model = Sequential()
    
    model.add(LSTM(128, input_shape=input_shape, return_sequences=True))
    model.add(Dropout(d))
        
    model.add(LSTM(128, input_shape=input_shape, return_sequences=False))
    model.add(Dropout(d))
        
    model.add(Dense(32,kernel_initializer="normal",activation='relu'))        
    model.add(Dense(1,kernel_initializer="normal",activation='linear'))
    
    model.compile(loss='mse',optimizer='adam', metrics=['accuracy'])
    
    return model

In [25]:
def nextDayPrediction(typeBlockchain, stock):    
    
    plot = True
    plotHictory = True
    interactiveGrapth = True
    plotForTrain = False
    
    df = get_data.get_data_frame(typeBlockchain, stock)
    
    df.index  = df.date
    df = df[[ 'open','close','low','high','volume']]

    df = ind.ma_rel_diff(df)
    df = ind.ema_rel_diff(df)
    df = ind.mom(df)
    df = ind.roc(df)
    df = ind.bbands(df)
    df = ind.normalized_bbands(df)
    df = ind.rsi(df)
    df = ind.stochastics(df)
    df = ind.macd(df)
    df = ind.atr(df)
    df = ind.adx(df)
    df = df.dropna()
    
    
    x_scaler = MinMaxScaler()
    y_scaler = MinMaxScaler()

    all_df = df.copy()
    
    f_list = [['high', 'volume', 'ma_rel_diff_50',
       'ema_rel_diff_10', 'moment_20', 'roc_14', 'boil_mean_20_2',
        'boil_down_20_2', 'normBB', 'rsi_14', 'stoch_d_14_3',
       'macd_12_26_9', 'atr_14', 'ATR', 'adx_14']]
    
    
    for features in f_list:
        
        x = all_df[features].copy()

        y = all_df['close'].copy()
        NUM_FEATURES = x.shape[1]
        
        x[features] = x_scaler.fit_transform(x)
        
        
        y = y_scaler.fit_transform(y.values.reshape(-1, 1))        
        x['close'] = y ####!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
        
        X_train, y_train, X_test, y_test = load.load_data(x, WINDOW, train_size= 0.97, TrainTest = True)

        x = all_df[features].copy() ####!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
        
        model = build_model(input_shape=(WINDOW, NUM_FEATURES))

        print('START FIT MODEL...')
        print(features)
        print()
        start = time.time()

        #history = History()
        #history= model.fit(X_train, y_train, validation_data=(X_test, y_test),  batch_size=32,\
        #                   epochs=405,verbose=0,
        #          callbacks=[history])

        #model.fit(X_train, y_train, batch_size=32, epochs=500, verbose=1)
        end = time.time()

        print ('Learning time: ', end-start)

        today = time.strftime("_%d_%m_%Y")
        pathModel = "./model_" + str(features) + typeBlockchain + today +".h5"
        save_model(model, pathModel)

        model = load_model(pathModel)
        lastbatch = np.array(x[-WINDOW:])
        pred = model.predict([lastbatch.reshape(1,WINDOW, NUM_FEATURES)])
        pred =  np.array(y_scaler.inverse_transform(pred)) # predicted value
        
        # one day prediction. get last batch known data (now we didnt need in y value and can predict it)    
        lastDate =str(df.last_valid_index()).split('-')
        currentData = datetime.date(int(lastDate[0]),int(lastDate[1]),int(lastDate[2])) + datetime.timedelta(1)
        predictionDate = pd.date_range(currentData, periods=1)
        prediction = pd.DataFrame(pred, columns=["predictionPrice"], index = predictionDate.values)


        print (prediction)



        if plotForTrain:

            trainPredict = model.predict(X_train)
            trainPredict = y_scaler.inverse_transform(trainPredict)
            prices = df.close.values.astype('float32')
            prices = prices.reshape(len(prices), 1)
            trainPredictPlot = np.empty_like(prices)
            trainPredictPlot[:, :] = np.nan
            trainPredictPlot[WINDOW:len(trainPredict)+WINDOW, :] = trainPredict
            Actual = pd.DataFrame(prices, columns=["close"], index=df.index).close
            Training = pd.DataFrame(trainPredictPlot, columns=["close"], index=df.date).close
            ActualValues = go.Scatter( x = df.date, y = Actual, name = 'ActualValues')
            TrainingValues = go.Scatter( x = df.date, y = Training, name = 'TrainingValues')

            iplot([ActualValues,TrainingValues])
            plt.show()

        if plot:
            calc_income = all_df.close[-y_test.shape[0]:]
            print (X_train.shape)
            trainPredict = model.predict(X_train)
            testPredict = model.predict(X_test)

            trainPredict = y_scaler.inverse_transform(trainPredict)
            trainY = y_scaler.inverse_transform([y_train])

            testPredict = y_scaler.inverse_transform(testPredict)
            m = testPredict
            testY = y_scaler.inverse_transform([y_test])

            trainScore = metrics.mean_squared_error(trainY[0], trainPredict[:,0]) ** .5
            print('Train Score: %.2f RMSE' % (trainScore))

            testScore = metrics.mean_squared_error(testY[0], testPredict[:,0]) ** .5
            print('Test Score: %.2f RMSE' % (testScore))



            prices = df.close.values.astype('float32')
            prices = prices.reshape(len(prices), 1)
            trainPredictPlot = np.empty_like(prices)
            trainPredictPlot[:, :] = np.nan
            trainPredictPlot[WINDOW:len(trainPredict)+WINDOW, :] = trainPredict

            testPredictPlot = np.empty_like(prices)
            testPredictPlot[:, :] = np.nan
            testPredictPlot[(len(prices) - testPredict.shape[0]):len(prices), :] = testPredict

            plt.plot(pd.DataFrame(prices, columns=["close"], index=df.index).close, label='Actual')
            plt.plot(pd.DataFrame(trainPredictPlot, columns=["close"], index=df.index).close, label='Training')
            plt.plot(pd.DataFrame(testPredictPlot, columns=["close"], index=df.index).close, label='Testing')
            plt.legend(loc='best')
            plt.show()

            interactiveGrapth = 1
            if interactiveGrapth:

                Actual = pd.DataFrame(prices, columns=["close"], index=df.index).close
                Training = pd.DataFrame(trainPredictPlot, columns=["close"], index=df.index).close
                Testing = pd.DataFrame(testPredictPlot, columns=["close"], index=df.index).close
                ActualValues = go.Scatter( x = df.index, y = Actual, name = 'ActualValues')
                TrainingValues = go.Scatter( x = df.index, y = Training, name = 'TrainingValues')
                TestingValues = go.Scatter( x = df.index, y = Testing, name = 'PredictedValues')

                iplot([ActualValues,TrainingValues, TestingValues])
                plt.show()

            if plotHictory:

                plt.plot(history.history['loss'], label = 'TrainLoss')
                plt.plot(history.history['val_loss'], label = 'TestLoss')
                plt.legend()
                plt.show()
            del model

            K.clear_session()

    
    return prediction